In [3]:
import numpy as np

In [217]:
N = 20
D = 5
Th = np.random.random((N, D))*2
x = np.random.random(D)

In [218]:
Th.dot(x)

array([ 3.45029072,  3.47077549,  1.85263096,  4.14024149,  2.31690248,
        2.88920979,  2.78723281,  3.58210333,  2.40598981,  3.71423114,
        4.27318994,  2.15629054,  4.59354189,  3.59212141,  3.53853952,
        4.04261468,  3.2722202 ,  2.91832432,  1.67406885,  2.76704534])

In [219]:
L = 4
thresholds = [1, 3, 5]

In [220]:
from math import exp

def sigmoid(x):
    return 1 / (1 + exp(-x))

sigma = np.vectorize(sigmoid)

In [221]:
cdf = sigma(np.outer(Th.dot(x), np.ones(3)) - thresholds)
#cdf

In [222]:
pdf = np.column_stack((np.ones(N), cdf))
for i in range(3):
    pdf[:, i] -= pdf[:, i + 1]
pdf

array([[ 0.07941729,  0.30987436,  0.43558009,  0.17512826],
       [ 0.07793249,  0.30650022,  0.43746011,  0.17810718],
       [ 0.29888124,  0.46014879,  0.1997749 ,  0.04119507],
       [ 0.04147752,  0.20079851,  0.46033417,  0.2973898 ],
       [ 0.2113341 ,  0.45309558,  0.27159219,  0.06397813],
       [ 0.13133459,  0.39633466,  0.36427826,  0.10805249],
       [ 0.14341232,  0.40957971,  0.34839813,  0.09860984],
       [ 0.07029914,  0.28814962,  0.44655971,  0.19499153],
       [ 0.19686734,  0.4474174 ,  0.28619035,  0.06952491],
       [ 0.06213881,  0.26652577,  0.45476558,  0.21656984],
       [ 0.03650247,  0.1822092 ,  0.4553932 ,  0.32589513],
       [ 0.23934197,  0.45990393,  0.24574671,  0.0550074 ],
       [ 0.0267647 ,  0.14212146,  0.43135214,  0.3997617 ],
       [ 0.0696472 ,  0.28650105,  0.44728287,  0.19656887],
       [ 0.07320019,  0.2953272 ,  0.44322856,  0.18824405],
       [ 0.04553739,  0.21510841,  0.46195219,  0.277402  ],
       [ 0.09344995,  0.

In [223]:
np.argmax(pdf, axis=1)

array([2, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1])

In [61]:
def logloss(x):
    return np.log(1 + np.exp(-x))

In [64]:
labels = np.round(np.random.random(N)*3)

In [74]:
labels

array([ 1.,  3.,  2.,  2.,  0.,  3.,  3.,  3.,  1.,  1.,  1.,  0.,  0.,
        2.,  1.,  1.,  0.,  1.,  2.,  2.])

In [90]:
sgn = np.fromfunction(lambda i, j: 2 * (1 + j < labels[i]) - 1, (N, L - 1), dtype=int)

In [92]:
delta = np.outer(Th.dot(x), np.ones(3)) - thresholds

In [181]:
logloss(sgn * delta).sum(axis=1)

array([ 2.27768573,  1.02478021,  1.0282167 ,  1.20785977,  2.21483259,
        1.23659014,  1.56966178,  1.7360527 ,  2.05443371,  1.44295324,
        2.00376447,  1.4911709 ,  1.7712483 ,  1.55937083,  1.29511481,
        0.81654143,  1.81876996,  2.08580664,  1.04565353,  1.2909893 ])

In [138]:
import tensorflow as tf

aa = tf.constant([1., 5., 3.])
a = tf.reshape(aa, (3, 1))
b = tf.ones([1, 3])
b.eval()

array([[ 1.,  1.,  1.]], dtype=float32)

In [135]:
sess = tf.InteractiveSession()
sess.run(tf.matmul(a, b))

array([[ 1.,  1.,  1.],
       [ 5.,  5.,  5.],
       [ 3.,  3.,  3.]], dtype=float32)

In [141]:
tf.transpose(aa).eval().shape

(3,)

In [189]:
logits = tf.constant(Th.dot(x))
thr = tf.cast(tf.constant(thresholds), tf.float64)
lab = tf.constant(labels)

In [190]:
lab

<tf.Tensor 'Const_37:0' shape=(20,) dtype=float64>

In [191]:
thr.eval()

array([ 1.,  2.,  4.])

In [192]:
logits

<tf.Tensor 'Const_35:0' shape=(20,) dtype=float64>

In [193]:
NB_CLASSES = 4

In [194]:
def tf_logloss(x):
    return tf.log(1 + tf.exp(-x))

In [195]:
def all_thresholds(labels, logits, thresholds):
    batch_size = tf.size(labels).eval()
    sgn = np.fromfunction(lambda i, j: 2 * (1 + j < labels.eval()[i]) - 1, (batch_size, NB_CLASSES - 1), dtype=int)
    delta = tf.matmul(tf.reshape(logits, (batch_size, 1)), tf.ones([1, NB_CLASSES - 1], tf.float64)) - thresholds
    return tf.reduce_sum(tf_logloss(sgn * delta), axis=1)

In [196]:
all_thresholds(lab, logits, thr).eval()

array([ 2.27768573,  1.02478021,  1.0282167 ,  1.20785977,  2.21483259,
        1.23659014,  1.56966178,  1.7360527 ,  2.05443371,  1.44295324,
        2.00376447,  1.4911709 ,  1.7712483 ,  1.55937083,  1.29511481,
        0.81654143,  1.81876996,  2.08580664,  1.04565353,  1.2909893 ])

In [197]:
delta

array([[ 1.10466859,  0.10466859, -1.89533141],
       [ 1.18095192,  0.18095192, -1.81904808],
       [ 0.20067352, -0.79932648, -2.79932648],
       [ 1.18566875,  0.18566875, -1.81433125],
       [ 1.05962956,  0.05962956, -1.94037044],
       [ 0.8505688 , -0.1494312 , -2.1494312 ],
       [ 0.45448788, -0.54551212, -2.54551212],
       [ 0.28697738, -0.71302262, -2.71302262],
       [ 0.94084964, -0.05915036, -2.05915036],
       [ 0.42024704, -0.57975296, -2.57975296],
       [ 0.9020796 , -0.0979204 , -2.0979204 ],
       [ 0.46637309, -0.53362691, -2.53362691],
       [ 0.71527384, -0.28472616, -2.28472616],
       [ 1.76366136,  0.76366136, -1.23633864],
       [ 0.27143715, -0.72856285, -2.72856285],
       [-0.32244028, -1.32244028, -3.32244028],
       [ 0.7547402 , -0.2452598 , -2.2452598 ],
       [ 0.96454238, -0.03545762, -2.03545762],
       [ 0.65705564, -0.34294436, -2.34294436],
       [ 1.35262482,  0.35262482, -1.64737518]])